In [1]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold
#warnings.filterwarnings('ignore')
#%matplotlib inline
from sklearn.metrics import roc_auc_score
## 数据降维处理的
from sklearn.model_selection import train_test_split  
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder


In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
sub=pd.read_csv("submission.csv")


In [3]:
data=pd.concat([train,test])
data['incident_date'] = pd.to_datetime(data['incident_date'],format='%Y-%m-%d')
startdate = datetime.datetime.strptime('2022-06-30', '%Y-%m-%d')
data['time'] = data['incident_date'].apply(lambda x: startdate-x).dt.days


In [4]:
#Encoder
numerical_fea = list(data.select_dtypes(include=['object']).columns)
division_le = LabelEncoder()
for fea in numerical_fea:
    division_le.fit(data[fea].values)
    data[fea] = division_le.transform(data[fea].values) 
print("数据预处理完成!")
testA=data[data['fraud'].isnull()].drop(['policy_id','incident_date','fraud'],axis=1)
trainA=data[data['fraud'].notnull()]
data_x=trainA.drop(['policy_id','incident_date','fraud'],axis=1)
data_y=train[['fraud']].copy()
col=['policy_state','insured_sex','insured_education_level','incident_type','collision_type','incident_severity','authorities_contacted','incident_state',
     'incident_city','police_report_available','auto_make','auto_model']
for i in data_x.columns:
    if i in col:
        data_x[i] = data_x[i].astype('str')
for i in testA.columns:
    if i in col:
        testA[i] = testA[i].astype('str')     

数据预处理完成!


In [5]:
  
model=CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            task_type="CPU",
            learning_rate=0.1,
            iterations=10000,
            random_seed=2020,
            od_type="Iter",
            depth=7,
            early_stopping_rounds=300)
answers = []
mean_score = 0
n_folds = 10
sk = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2019)
for train, test in sk.split(data_x, data_y):
    x_train = data_x.iloc[train]
    y_train = data_y.iloc[train]
    x_test = data_x.iloc[test]
    y_test = data_y.iloc[test]
    clf = model.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=500,cat_features=col)
    yy_pred_valid=clf.predict(x_test)
    print('cat验证的auc:{}'.format(roc_auc_score(y_test, yy_pred_valid)))
    mean_score += roc_auc_score(y_test, yy_pred_valid) / n_folds
    y_pred_valid = clf.predict(testA,prediction_type='Probability')[:,-1]
    answers.append(y_pred_valid)
print('10折平均Auc:{}'.format(mean_score))
lgb_pre=sum(answers)/n_folds
sub['fraud']=lgb_pre
sub.to_csv('金融2预测.csv',index=False) 

0:	test: 0.7564103	best: 0.7564103 (0)	total: 207ms	remaining: 34m 32s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8563034188
bestIteration = 2

Shrink model to first 3 iterations.
cat验证的auc:0.7008547008547008
0:	test: 0.6997863	best: 0.6997863 (0)	total: 50.5ms	remaining: 8m 25s
500:	test: 0.8514957	best: 0.8696581 (266)	total: 43.1s	remaining: 13m 38s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8696581197
bestIteration = 266

Shrink model to first 267 iterations.
cat验证的auc:0.764957264957265
0:	test: 0.8782051	best: 0.8782051 (0)	total: 71.2ms	remaining: 11m 51s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.9476495726
bestIteration = 29

Shrink model to first 30 iterations.
cat验证的auc:0.8226495726495727
0:	test: 0.7467949	best: 0.7467949 (0)	total: 77ms	remaining: 12m 50s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8397435897
bestIteration = 5

Shrink model to first 6 iterations.
cat验证的auc